
# Iris Classification with Logistic Regression (FastAPI Project Prep)

This notebook demonstrates:
- Loading the Iris dataset (built-in sklearn)
- Validating data quality (missing, duplicates, stats)
- Training a Pipeline (StandardScaler + Logistic Regression)
- Evaluating the model
- Saving the model (`model.pkl`) and metadata (`model_info.json`)
- Making sample predictions


In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import joblib
import json
from datetime import datetime

In [2]:
iris = load_iris()
X = iris['data']
y = iris['target']
feature_names = iris['feature_names']
class_names = iris['target_names'].tolist()

df = pd.DataFrame(X, columns=feature_names)
df['target'] = [class_names[i] for i in y]

print("First 5 rows of dataset:")
df.head()

First 5 rows of dataset:


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [3]:
# Data Validation / Cleaning
print("\nMissing values per column:")
print(df.isnull().sum())

print("\nNumber of duplicate rows:", df.duplicated().sum())

print("\nBasic statistics:")
df.describe()


Missing values per column:
sepal length (cm)    0
sepal width (cm)     0
petal length (cm)    0
petal width (cm)     0
target               0
dtype: int64

Number of duplicate rows: 1

Basic statistics:


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [4]:
# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [5]:
# Build & Train Model
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("lr", LogisticRegression(max_iter=1000, multi_class="auto"))
])
clf.fit(X_train, y_train)

C:\Users\sheha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


,steps,"[('scaler', ...), ('lr', ...)]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0


In [6]:
# Evaluation
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=class_names)

print(f"\nTest Accuracy: {acc:.4f}")
print("\nClassification Report:\n", report)


Test Accuracy: 0.9333

Classification Report:
               precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       0.90      0.90      0.90        10
   virginica       0.90      0.90      0.90        10

    accuracy                           0.93        30
   macro avg       0.93      0.93      0.93        30
weighted avg       0.93      0.93      0.93        30



In [7]:
# Save Artifacts
joblib.dump(clf, "model.pkl")

info = {
    "model_type": "Pipeline(StandardScaler -> LogisticRegression)",
    "problem_type": "classification",
    "created_at": datetime.utcnow().isoformat() + "Z",
    "features": feature_names,
    "classes": class_names,
    "metrics": {
        "test_accuracy": acc
    }
}
with open("model_info.json", "w") as f:
    json.dump(info, f, indent=2)

print("\nArtifacts saved: model.pkl, model_info.json")


Artifacts saved: model.pkl, model_info.json


C:\Users\sheha\AppData\Local\Temp\ipykernel_15324\4046331554.py:7: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "created_at": datetime.utcnow().isoformat() + "Z",


In [9]:
# Sample Prediction
sample = np.array([[5.1, 3.5, 1.4, 0.2]])  # expected setosa
pred_idx = clf.predict(sample)[0]
proba = clf.predict_proba(sample)[0]

print("\nSample Input:", sample.tolist())
print("Prediction:", class_names[pred_idx])
print("Probabilities:", {class_names[i]: float(proba[i]) for i in range(len(proba))})


Sample Input: [[5.1, 3.5, 1.4, 0.2]]
Prediction: setosa
Probabilities: {'setosa': 0.9808127381969137, 'versicolor': 0.019186992121989523, 'virginica': 2.6968109686920263e-07}
